In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
plt.style.use('seaborn-notebook')
%matplotlib inline

In [ ]:
name = 'E3_Rurbanos'
df = gpd.read_file('../data/interim/shapes/FBio_' + name + '.shp')
df = df[df.geometry.notnull()].to_crs({'init': 'epsg:4326'}) 
print (len(df))

In [ ]:
df.columns

In [ ]:
df.head()

In [ ]:
states = gpd.read_file('../data/interim/shapes/mex-states.shp')
# Reproject the file
states = states.to_crs({'init': 'epsg:4326'}) 

In [ ]:
import json, os
with open(os.path.join('../data/interim/', 'trans-regions.json'), 'r') as fp:
    trans_regions = json.load(fp)

In [ ]:
loadzones = gpd.read_file('../data/interim/shapes/Mask_T.shp')
loadzones = loadzones.to_crs({'init': 'epsg:4326'}) 
loadzones['trans-region'] = loadzones['ID'].astype(int).map('{0:02}'.format).map(trans_regions)

In [ ]:
assert loadzones.crs == df.crs

In [ ]:
# Base plot
base = states.plot(color='white', edgecolor='black')
loadzones.plot(ax=base, color='white', edgecolor='blue', label='Loadzones')
df.plot(color='red', ax=base, label='Outside', markersize=5)
if not 'forestales' in name:
    join = gpd.sjoin(df, loadzones, op='within')
    join.plot(ax=base, color='green', label='Inside', markersize=5)
else:
    join = gpd.overlay(loadzones, df, how='intersection')
    join.plot(ax=base, color='green', legend=True)
plt.title(name)
plt.xlabel('lng')
plt.ylabel('lat')
plt.legend();
plt.savefig(name + '.png', dpi=300)

In [ ]:
# Create data
os.makedirs('data', exist_ok=True)
try: 
    columns = ['trans-region', 'X', 'Y', 'CLASIFICAC', 'TIPO', 'PROCESO', 
               'GENE_GWha', 'CAPINST_MW', 'FP',]
    bioenergy = join[columns].copy();
except KeyError:
    columns = ['LATITUD', 'LONGITUD', 'trans-region', 'CLASIFICAC', 'TIPO', 'PROCESO', 
               'GENE_GWha', 'CAPINST_MW', 'FP']
    bioenergy = join[columns].copy();


bioenergy['CLASIFICAC'] = bioenergy.CLASIFICAC.map(str.lower).str.replace(' ', '_')
bioenergy['TIPO'] = bioenergy.TIPO.map(str.lower).str.replace(' ', '_')
bioenergy['PROCESO'] = bioenergy.PROCESO.map(str.lower).str.replace(' ', '_')
if 'E3' in name:
    scenario = 'high'
else:
    scenario = 'low'
bioenergy.loc[:, 'scenario'] = scenario
bioenergy = bioenergy.rename(columns={'X': 'lng','LATITUD':'lat', 'LONGITUD': 'lng', 'Y': 'lat', 'CLASIFICAC': 'source',
                         'TIPO': 'category', 'FP': 'cf',
                         'GENE_GWha': 'gen_GWha', 'CAPINST_MW':'cap_MW',
                         'PROCESO': 'fuel_type'})

In [ ]:
bioenergy.to_csv('data/{}.csv'.format(name), index=False)

In [ ]:
from itertools import zip_longest

#join = gpd.sjoin(df, loadzones, op='within')
load_zones_out = loadzones[~loadzones['trans-region']
                          .isin(join['trans-region'].unique())]['trans-region']
load_zones_in = join.groupby('trans-region')['geometry'].count().index.values
#join = gpd.overlay(loadzones, df, how='intersection')
with open(name + '.txt', 'w') as fpd:
    fpd.write('Trans regions included:\t{0}\n'.format(len(join['trans-region'].unique())))
    fpd.write('Trans regions not included: \t{0}\n'
              .format(len(loadzones[~loadzones['trans-region']
                                    .isin(join['trans-region'].unique())])))
    fpd.write('Total points:\t{0}\n'.format(len(df)))
    fpd.write('Points inside:\t{0}\n'.format(len(join)))
    fpd.write('Points outside:\t{0}\n'.format(len(df) - len(join)))
    fpd.write('Trans_in\tTrans_out\n')
    for values in zip_longest(*[load_zones_in, load_zones_out]):
        fpd.write('{0}\t{1}\n'.format(values[0],values[1]))